In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import fnmatch
import os
import kuka_utils as u


In [2]:
#LATEX FORMAT PRINTING NUMBER OF MOVING ECHOES IN EACH AREA HH and VH WITH TOTALS

#using full bandwith data, so use deconvolved
decon = ''

date_list = [
             '20191107',
            '20191114',
            '20191220',
            '20191226',
            '20200102',
            '20200109',
            '20200112',
            '20200116',
            '20200119',
            '20200123',
            '20200124',
            '20200130', 
            ]

areas = u.areas()
yrange = [1,3]
pols = ['hh','vh']
band = ['Ku', 'Ka']
dpi = 600
area_list = [keys for keys in areas]

#set up dict for totals
area_totals = {}
for b in band:
    area_totals[b] = {}
    for a in area_list:
        area_totals[b][a] = {}
        area_totals[b][a]['total'] = 0
        for p in ['hh','vh']:
            area_totals[b][a][p] = {'max_at_as':0}

for date in date_list:
    #string to contain the values for printing
    string = np.full(10, '    ')
    
    for b in band:

        if b == 'Ku':
            threshold = -50
            up_down = .1
            budge = 0  
        elif b == 'Ka':
            threshold = -55
            up_down = 0.06
            budge = 1 #this shifts it over one column for the Ka-band values
            
        
        file = 'x_y_p_'+date+'_'+b+'.p'
        if (len(fnmatch.filter(os.listdir('/Volumes/bigLacie/processed_stare_new_nrcs/x_y_p/'), file))) == 1:
#             print(1)
            x_y_p = pickle.load( open('/Volumes/bigLacie/processed_stare_new_nrcs/'  + '/x_y_p/x_y_p_'+date+'_'+b+'.p', "rb" ) )
#             print(2)
            width = round(up_down/(x_y_p['range'][1] - x_y_p['range'][0])) #width of window to look for air snow interface

            #find the range bins at the min and max of yrange
            r0 = np.where(abs(x_y_p['range'] - min(yrange)) == min(abs(x_y_p['range'] - min(yrange))))[0][0]
            r1 = np.where(abs(x_y_p['range'] - max(yrange)) == min(abs(x_y_p['range'] - max(yrange))))[0][0]

            #get the co and cross pol data peaks
    #                         print('r0', r0, 'r1', r1, 'index[a]', index[a], 'moving', moving)
            hh = np.array(x_y_p['hh'])[r0:r1]
            max_peak_hh = [np.where(hh[:,i] == max(hh[:,i]))[0][0] for i in np.arange(hh.shape[1])] + r0

            vh = np.array(x_y_p['vh'])[r0:r1]
            max_peak_vh = [np.where(vh[:,i] == max(vh[:,i]))[0][0] for i in np.arange(vh.shape[1])] + r0

            index = u.index_areas(x_y_p['x'], x_y_p['y'])

            for a in areas:

                area_totals[b][a][date] = {}
                aindex = np.where(np.array(area_list) == a)[0][0]
                
                if len(index[a]) == 0:
                    string[aindex*2 + budge] = '-'

                elif len(index[a]) > 0:
                    dist, moving = u.remove_static(np.array(x_y_p['x'])[index[a]], 
                                                   np.array(x_y_p['y'])[index[a]], 
                                                   cutoff = .05)
                    
                    area_totals[b][a][date]['moving'] = len(moving)

                    if len(moving) > 0:
                        
                        area_totals[b][a][date]['max_peak_hh'] = max_peak_hh[index[a][moving]]
                        area_totals[b][a][date]['max_peak_vh'] = max_peak_vh[index[a][moving]]
 
                        as_int = np.full(len(moving), np.nan)
#                         print(3)
        
                        #get the waveforms at range bins of interest
                        wf = np.array(x_y_p['hh'+decon])[r0:r1,index[a][moving]] 
                
                        #start bin to look for air/snow interface where power > threshold
                        sta = np.array([np.where(wf[:,i] > 10**(threshold/10))[0][0]
                                    if max(wf[:,i]) > 10**(threshold/10) 
                                    else -9999
                                    for i in range(len(moving))]) 
                        
                        #stop bin for air/snow interface search
                        sto = sta + width 

                        # find highest amplitude peak in the window between sta and sto
                        as_int = [np.where((wf[sta[i]:sto[i], i] == max(wf[sta[i]:sto[i], i])) #highest power 
                                & (wf[sta[i]:sto[i], i] > np.roll(wf[sta[i]:sto[i], i], 1)) #and also a maximum
                                & (wf[sta[i]:sto[i], i] > np.roll(wf[sta[i]:sto[i], i], -1)))[0][0] + sta[i] + r0
                                if sta[i] > 0 #check not a nan, i.e. the power exceeds threshold somewhere in the waveform
                                else np.nan
                                for i in range(len(moving))]

#                         for i in range(len(moving)):
            
#                             e = np.array(x_y_p['hh'+decon])[r0:r1,index[a][moving][i]]
                    
#                             if max(e > 10**(threshold/10)):
#                                 as_start = np.where(e > 10**(threshold/10))[0][0]
#                                 search = np.array(e[as_start:as_start + width])
#                                 as_int_i = np.where((search == max(search)) & ( search > np.roll(search, -1)) & (search > np.roll(search, 1)))
# #                                 print('as_int_i', as_int_i)
#                                 if len(as_int_i[0]) == 1:
#                                     as_int[i] = as_int_i[0][0] + r0 + as_start

                        
#                             plt.plot(x_y_p['range'][r0:r1], 10*np.log10(e), 'k-')
                            
#                             plt.plot(x_y_p['range'][as_int[i].astype(int)], 
#                                      10*np.log10(e[as_int[i].astype(int)-r0]), 
#                                      'r.', markersize = 10)
        
#                             plt.plot(x_y_p['range'][max_peak_hh[index[a][moving]][i].astype(int)], 
#                                      10*np.log10(e[max_peak_hh[index[a][moving]][i].astype(int)-r0]),
#                                      'c.', markersize = 3) 
            
#                             plt.show()
                
#                         print(4)
                        area_totals[b][a][date]['as_int'] = as_int

                        max_at_as_hh = len(np.where(as_int == area_totals[b][a][date]['max_peak_hh'])[0])
                        max_at_as_vh = len(np.where(as_int == area_totals[b][a][date]['max_peak_vh'])[0])

                        string[aindex*2 + budge] = str(int(round(max_at_as_hh/len(as_int)*100,0))) + '/' + str(int(round(max_at_as_vh/len(as_int)*100,0)))

                        area_totals[b][a]['hh']['max_at_as'] += max_at_as_hh
                        area_totals[b][a]['vh']['max_at_as'] += max_at_as_vh
                        area_totals[b][a]['total'] += len(as_int)
                    else:
                        string[aindex*2 + budge] = '-'
        else:
            for a in areas:
                aindex = np.where(np.array(area_list) == a)[0][0]
                string[aindex*2 + budge] = '-'

    a = ''
    for s in string:
        a+=('&' + s)
    print(date + a + r'\\')
    
s = []
ku_hh = 0
ku_vh = 0
ku_asi = 0
ka_hh = 0
ka_vh = 0
ka_asi = 0

for a in area_list:
    if area_totals['Ku'][a]['total'] > 0:
        s += str(int(round(area_totals['Ku'][a]['hh']['max_at_as']/area_totals['Ku'][a]['total']*100))) + '/'
        s += str(int(round(area_totals['Ku'][a]['vh']['max_at_as']/area_totals['Ku'][a]['total']*100))) + '&'
        s += str(int())
        ku_hh += area_totals['Ku'][a]['hh']['max_at_as']
        ku_vh += area_totals['Ku'][a]['vh']['max_at_as']
        ku_asi += area_totals['Ku'][a]['total']
    else:
        s += '-&'
    if area_totals['Ka'][a]['total'] > 0:
        s += str(int(round(area_totals['Ka'][a]['hh']['max_at_as']/area_totals['Ka'][a]['total']*100))) + '/'
        s += str(int(round(area_totals['Ka'][a]['vh']['max_at_as']/area_totals['Ka'][a]['total']*100))) + '&'
        ka_hh += area_totals['Ka'][a]['hh']['max_at_as']
        ka_vh += area_totals['Ka'][a]['vh']['max_at_as']
        ka_asi += area_totals['Ka'][a]['total']

    else:
        s += '-&'

r = ''.join([str(item) for item in s])
print('Total&' + r + '\\')

print(' ')
print('Ku HH % as', (ku_hh / ku_asi)*100)
print('Ka HH % as', (ka_hh / ka_asi)*100)
print('Ku VH % as', (ku_vh / ku_asi)*100)
print('Ka VH % as', (ka_vh / ka_asi)*100)



20191107&58/0&75/0&-&-&-&-&84/2&86/0&-&-\\
20191114&74/0&79/0&-&-&-&-&90/4&86/1&-&-\\
20191220&64/0&64/0&-&-&-&-&-&-&-&-\\
20191226&68/0&58/0&80/2&79/0&-&-&-&-&-&-\\
20200102&78/0&74/0&84/1&85/1&-&-&-&-&-&-\\
20200109&61/0&59/0&70/1&76/0&-&-&-&-&-&-\\
20200112&-&-&-&-&69/3&-&-&-&-&-\\
20200116&79/0&76/0&84/2&86/0&-&-&-&-&-&-\\
20200119&-&-&-&-&73/5&81/2&-&-&-&-\\
20200123&-&-&-&-&-&-&-&-&-&-\\
20200124&-&-&79/1&79/0&-&-&-&-&70/1&66/0\\
20200130&75/0&68/0&80/1&77/0&-&-&-&-&-&-\\
Total&71/0&067/0&79/1&080/0&71/4&081/2&87/3&086/0&70/1&066/0&\
 
Ku HH % as 74.86303416674967
Ka HH % as 73.97260273972603
Ku VH % as 1.1256101205299331
Ka VH % as 0.26989866069155166
